In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:

from datasets import load_dataset
import datasets

In [3]:
from transformers import AutoTokenizer

model_checkpoint = "deepset/roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
tokenizer

RobertaTokenizerFast(name_or_path='deepset/roberta-base-squad2', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [5]:
tokenizer.is_fast


True

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
import json
from random import shuffle

# Read JSON file
with open('insurance.json', 'r') as file:
    json_data = json.load(file)



In [8]:
count = 0
for obj in json_data:
    # Check if 'answers' key exists
    if 'answers' in obj:
      count+=1
print(count)


426


In [9]:
len(json_data)

427

In [10]:
# Filter out objects without the 'answers' key
filtered_data = [obj for obj in json_data if 'answers' in obj]



In [11]:
len(filtered_data)

426

In [12]:
# Iterate through the filtered data and convert 'answer_start' to a list
for obj in filtered_data:
    if not isinstance(obj['answers'][0].get('answer_start', []), list):
        obj['answers'][0]['answer_start'] = [obj['answers'][0]['answer_start']]


In [13]:
# Iterate through the filtered data and convert 'text' to a list
for obj in filtered_data:
    if not isinstance(obj['answers'][0].get('text', []), list):
        obj['answers'][0]['text'] = [obj['answers'][0]['text']]

In [18]:
# Shuffle the JSON objects randomly
shuffle(filtered_data)

# Calculate the split indices for data1 and data2
split_index = int(0.8 * len(filtered_data))
print(split_index)

# Split the JSON objects into data1 and data2
data1 = filtered_data[:split_index]
data2 = filtered_data[split_index:]

# Print the lengths of data1 and data2
print("Length of data1:", len(data1))
print("Length of data2:", len(data2))


340
Length of data1: 340
Length of data2: 86


In [20]:
# Create a dictionary with column names and values
data_dict1 = {
    "id": [d["id"] for d in data1],
    "title": [d["title"] for d in data1],
    "context": [d["context"] for d in data1],
    "question": [d["question"] for d in data1],
    "answers": [d["answers"] for d in data1]
}

In [21]:
# Create a dictionary with column names and values
data_dict2 = {
    "id": [d["id"] for d in data2],
    "title": [d["title"] for d in data2],
    "context": [d["context"] for d in data2],
    "question": [d["question"] for d in data2],
    "answers": [d["answers"] for d in data2]
}

In [22]:
from datasets import Dataset, DatasetDict


In [23]:
dataset1 = Dataset.from_dict(data_dict1)


In [24]:
dataset2 = Dataset.from_dict(data_dict2)


In [25]:
dataset_dict = DatasetDict({"train": dataset1, "test": dataset2})


In [26]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 340
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 86
    })
})

In [27]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [28]:
show_random_elements(dataset_dict["train"])

,id,title,context,question,answers
0,1,,Wyoming has the previous version of the model in WYO. STAT. ANN. §§ 26-5-112 to 26-5-117 (1992/2017).,Which state has the previous version of the model in WYO. STAT. ANN. §§ 26-5-112 to 26-5-117?,"[{'answer_start': [0], 'text': ['Wyoming']}]"
1,5,,Subsection D or E of this section shall not be allowed unless the assuming i nsurer a grees in the trust agreements to the following conditions: (2) The assets shall b e distributed by and claims s hall be filed with and valued by the commissioner with regulatory oversight in accorda nce with the laws of the state in which the tru st is domiciled that are applicable to the liquidation of domestic insurance companies.,What actions are taken with regard to the assets and claims under the commissioner's oversight?,"[{'answer_start': [66], 'text': ['distributed, filed, and valued']}]"
2,7,,C. A representative of a British banking,,"[{'answer_start': [], 'text': ['']}]"
3,4,,"I. If the assuming ins urer does not meet the requirements of Subsections A, B, C or F, the credit permitted by","What happens if the assuming insurer does not meet the requirements of Subsections A, B, C, or F?","[{'answer_start': [74], 'text': ['credit permitted is not provided']}]"
4,3,,"Section [ ] Credit Allowed a Foreign Ceding Insurer [Optional] As early as August 1993, the chair of the working group considering credit for reinsurance issues received a suggestion to add a new section to the model act that would allow credit to a foreign ceding insurer to the extent credit had been allowed by the ceding insurer’s state of domicile.",When did the chair of the working group receive a suggestion to add a new section?,"[{'answer_start': [0], 'text': ['As early as August 1993']}]"
5,1,,The certified reinsurer shall maintain separate trust accounts for its obligations incurred under reinsurance agreements issued or renewed as a certified reinsurer.,What type of accounts should the certified reinsurer maintain?,"[{'answer_start': [0], 'text': ['Separate trust accounts']}]"
6,2,,Amendments to this Act were made during the 2016 1st quarter of the NAIC Proceedings.,When were amendments made to this Act?,"[{'answer_start': [36], 'text': ['2016 1st quarter']}]"
7,5,,The information provided in the chart does not constitute a formal legal opinion by the NAIC staff.,Does the information provided in the chart constitute a formal legal opinion?,"[{'answer_start': [61], 'text': ['No']}]"
8,3,,It was important to be certain that language drafted was not worded so broadly as to permit any group which included individuals and had established a trust fund in the amount of $100 million to qualify as an accredited reinsurer and thereby circumvent other applicable requirements.,What was important to ensure in the language drafted?,"[{'answer_start': [15], 'text': ['Not to permit any group to qualify as an accredited reinsurer by wording the language too broadly']}]"
9,4,,"The trusteed surplus requirement of $20,000,000 is an option available to assuming insurers that do not",What is the trusteed surplus requirement for assuming insurers?,"[{'answer_start': [0], 'text': ['$20,000,000']}]"


In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [30]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [31]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [32]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [0, 2264, 16, 110, 766, 116, 2, 2, 2387, 766, 16, 28856, 1851, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [33]:
for i, example in enumerate(dataset_dict["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = dataset_dict["train"][i]

In [34]:
len(tokenizer(example["question"], example["context"])["input_ids"])

33

In [35]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

33

In [36]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [37]:
[len(x) for x in tokenized_example["input_ids"]]

[33]

In [38]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

<s>What must the assuming insurer's supervisory authority confirm?</s></s>The assuming insurer's supervisory authority must confirm compliance with certain requirements on an annual basis.</s>


In [39]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 9), (10, 13), (14, 22), (23, 30), (30, 32), (33, 38), (38, 44), (45, 54), (55, 62), (62, 63), (0, 0), (0, 0), (0, 3), (4, 12), (13, 20), (20, 22), (23, 28), (28, 34), (35, 44), (45, 49), (50, 57), (58, 68), (69, 73), (74, 81), (82, 94), (95, 97), (98, 100), (101, 107), (108, 113), (113, 114), (0, 0)]


In [40]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

What What


In [41]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [42]:
answers = example["answers"]
start_char = answers[0]["answer_start"][0]
end_char = start_char + len(answers[0]["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

15 20


In [43]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers[0]["text"][0])

 assuming insurer's supervisory authority
Compliance with certain requirements


In [44]:
pad_on_right = tokenizer.padding_side == "right"

In [45]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers[0]["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers[0]["answer_start"][0]
            end_char = start_char + len(answers[0]["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [46]:
features = prepare_train_features(dataset_dict['train'][:5])

In [47]:
tokenized_datasets = dataset_dict.map(prepare_train_features, batched=True, remove_columns=dataset_dict["train"].column_names)

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Map:   0%|          | 0/86 [00:00<?, ? examples/s]

In [48]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [49]:
batch_size = 16

In [87]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"finetuned-{model_name}-model-for-insurance-data",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    push_to_hub=True,
)

In [88]:
from transformers import default_data_collator

data_collator = default_data_collator

In [89]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/sprateek/fine_tuned-roberta-base-squad2-model-for-insurance-data into local empty directory.


In [90]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.113623
2,No log,3.395584
3,No log,3.511177


TrainOutput(global_step=66, training_loss=0.9354053844105114, metrics={'train_runtime': 69.4835, 'train_samples_per_second': 14.68, 'train_steps_per_second': 0.95, 'total_flos': 199892018903040.0, 'train_loss': 0.9354053844105114, 'epoch': 3.0})

In [60]:
trainer.save_model("insurance-trained3")

Upload file pytorch_model.bin:   0%|          | 1.00/473M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.93k [00:00<?, ?B/s]

Upload file runs/Jun14_08-21-36_729579a09a23/events.out.tfevents.1686730911.729579a09a23.283.0:   0%|         …

To https://huggingface.co/sprateek/finetuned-roberta-base-squad2-model-for-insurance-data
   585af1f..4a0643a  main -> main

   585af1f..4a0643a  main -> main

To https://huggingface.co/sprateek/finetuned-roberta-base-squad2-model-for-insurance-data
   4a0643a..00c55ec  main -> main

   4a0643a..00c55ec  main -> main



In [61]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [62]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [63]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([27, 10, 18, 33, 29, 15, 16, 33, 21, 11, 23, 16, 20, 22, 13, 14],
        device='cuda:0'),
 tensor([49, 21, 23, 36, 34, 19, 30, 35, 32, 20, 28, 30, 23, 24, 50, 19],
        device='cuda:0'))

In [64]:
n_best_size = 20

In [65]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [66]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [67]:
validation_features = dataset_dict["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset_dict["test"].column_names
)

Map:   0%|          | 0/86 [00:00<?, ? examples/s]

In [68]:
raw_predictions = trainer.predict(validation_features)

In [69]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [70]:
max_answer_length = 30

In [71]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset_dict["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 8.73712,
  'text': 'current model law requirement that alien reinsurers fund an amount equal to U.S. liabilities plus $20 million did'},
 {'score': 8.529247,
  'text': 'current model law requirement that alien reinsurers fund an amount equal to U.S. liabilities plus $20 million did not'},
 {'score': 8.373995,
  'text': 'interested party opined that the current model law requirement that alien reinsurers fund an amount equal to U.S. liabilities plus $20 million did'},
 {'score': 8.239497,
  'text': 'current model law requirement that alien reinsurers fund an amount equal to U.S. liabilities plus $20 million did not leave'},
 {'score': 8.229905,
  'text': 'opined that the current model law requirement that alien reinsurers fund an amount equal to U.S. liabilities plus $20 million did'},
 {'score': 8.166122,
  'text': 'interested party opined that the current model law requirement that alien reinsurers fund an amount equal to U.S. liabilities plus $20 million did not'},
 {'scor

In [72]:
dataset_dict["test"][0]["answers"]

[{'answer_start': [17],
  'text': ['It did not leave a very substantial margin for unrecoverability of obligations']}]

In [73]:
trainer.push_to_hub()

In [75]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("sprateek/finetuned-roberta-base-squad2-model-for-insurance-data")

In [81]:
context = "Credit shall be allowed under Subsections A, B or C of this section only as respects cessions of those kinds or classes of business which the assuming insurer is licensed or otherwise permitted to write or assume in its state of domicile or, in the case of a U.S. branch of an alien assuming insurer, in the state through which it is entered and licensed to transact insurance or reinsurance. Credit shall be allowed under Subsections C or D of this section only if the applicable requirements of Subsection H have been satisfied."
question = "When credit shall be allowed under which subsections? "


In [77]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="sprateek/finetuned-roberta-base-squad2-model-for-insurance-data")


{'score': 0.01414297241717577,
 'start': 8,
 'end': 80,
 'answer': 'trust fund for a single assuming insurer shall consist of funds in trust'}

In [82]:
question_answerer(question=question, context=context)

{'score': 0.1405206024646759,
 'start': 0,
 'end': 51,
 'answer': 'Credit shall be allowed under Subsections A, B or C'}